# Extraccion Datos El pais
### Detección automática de sarcasmos en textos utilizando Transformers
#### Ernesto Francisco Barrionuevo Aragonés 21727751

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

# Ruta al controlador de Chrome
PATH = 'chromedriver.exe'

# Configuración del servicio de Chrome
cService = ChromeService(executable_path=PATH)

# Función para aceptar el consentimiento
def accept_consent(driver):
    try:
        # Encuentra el nuevo botón de consentir usando el enlace (<a>) y la clase 'pmConsentWall-button'
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='pmConsentWall-button']"))
        )
        button.click()
        print("Se hizo clic en el botón 'Aceptar y continuar'.")
        time.sleep(3)  # Espera para asegurarse de que la acción se complete
    except Exception as e:
        print(f"No se encontró el botón 'Aceptar y continuar' o no se pudo hacer clic: {str(e)}")

# Función para extraer títulos
def extract_titles(url):
    titles = []
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        section = soup.find('div', {'id': 'results-container', 'data-dtm-region': 'buscador_resultados_noticias'})
        if section:
            articles = section.find_all('h2', class_='c_t')
            for article in articles[:15]:
                titles.append(article.get_text(strip=True))
        else:
            print("No se encontró la sección especificada.")
    else:
        print(f"No se pudo acceder a la página. Código de estado: {response.status_code}")
    return titles

# Función principal para realizar las búsquedas en bucle
def main():
    # Leer las palabras clave desde un archivo
    with open('keywords.txt', 'r', encoding='utf-8') as file:
        keywords = [line.strip() for line in file.readlines()]

    # Inicia el navegador Chrome
    driver = webdriver.Chrome(service=cService)
    base_url = "https://elpais.com/buscador/"

    # Aceptar el consentimiento de cookies una vez
    driver.get(base_url)
    accept_consent(driver)

    all_titles = []

    for keyword in keywords:
        search_url = f"{base_url}{keyword}/"
        driver.get(search_url)

        # Extraer títulos de la página actual
        titles = extract_titles(search_url)
        for title in titles:
            all_titles.append({"keyword": keyword, "title": title})

        # Espera unos segundos antes de pasar a la siguiente búsqueda
        time.sleep(2)

    # Cierra el navegador
    driver.quit()

    # Guardar los resultados en un archivo CSV
    df = pd.DataFrame(all_titles)
    df.to_csv('resultados.csv', index=False)

if __name__ == "__main__":
    main()



Se hizo clic en el botón 'Aceptar y continuar'.
No se encontró la sección especificada.
No se pudo acceder a la página. Código de estado: 500
No se encontró la sección especificada.
No se pudo acceder a la página. Código de estado: 500
No se pudo acceder a la página. Código de estado: 500
No se pudo acceder a la página. Código de estado: 500
No se pudo acceder a la página. Código de estado: 500
No se pudo acceder a la página. Código de estado: 500
No se encontró la sección especificada.
No se encontró la sección especificada.
No se encontró la sección especificada.
No se pudo acceder a la página. Código de estado: 500
No se encontró la sección especificada.
No se encontró la sección especificada.
